# Imports

In [ ]:
import pandas as pd
import re

# Chargement des données

In [ ]:
# Création d'un dossier appelé data
!mkdir data

# Téléchargement du fichier contenant les fanfictions et leurs métadonnées
!wget -P data https://gitlab.unistra.fr/paulinemoreau/memoire/-/raw/main/data/vrac/pretraitements/valide/fanfics_clean.csv

--2025-03-25 18:01:02--  https://gitlab.unistra.fr/paulinemoreau/memoire/-/raw/main/data/vrac/pretraitements/valide/fanfics_clean.csv
Resolving gitlab.unistra.fr (gitlab.unistra.fr)... 130.79.254.48
Connecting to gitlab.unistra.fr (gitlab.unistra.fr)|130.79.254.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 195674897 (187M) [text/plain]
Saving to: ‘data/fanfics_clean.csv’

fanfics_clean.csv   100%[===================>] 186.61M  24.1MB/s    in 8.6s    

2025-03-25 18:01:11 (21.7 MB/s) - ‘data/fanfics_clean.csv’ saved [195674897/195674897]



In [ ]:
df_fanfic = pd.read_csv('data/fanfics_clean.csv')

In [ ]:
df_fanfic.head()

,work_id,tag,title,author,rating,category,fandom,relationship,character,additional tags,...,chapters,comments,kudos,bookmarks,hits,all_kudos,all_bookmarks,body,body_clean,nb_caracteres
0,63563206,Fluff,Nuit etoile,['Lady_Meg_666'],General Audiences,M/M,Harry Potter - J. K. Rowling,Sirius Black/Remus Lupin,"Sirius Black, Remus Lupin","Established Relationship, Fluff, Domestic Fluf...",...,1/1,1.0,NaN,NaN,14,[],[],"<< Regarde, murmure Sirius en montrant le ciel...","<< Regarde, murmure Sirius en montrant le ciel...",1413
1,63528274,Fluff,Je le deteste,['wilk'],Not Rated,M/M,Hawaii Five-0 (2010),"Steve McGarrett/Danny ""Danno"" Williams","Danny ""Danno"" Williams, Steve McGarrett",Fluff,...,1/1,1.0,5.0,NaN,59,"['MommaJ', 'WhiteShiro']",[],Danny deteste vraiment Steve : a peine deux jo...,Danny deteste vraiment Steve : a peine deux jo...,600
2,63513724,Fluff,Camomille,['MyLovelyKitchenSinkDrama'],General Audiences,M/M,Guns N' Roses,Axl Rose/Izzy Stradlin,"Izzy Stradlin, Axl Rose","Domestic Fluff, Self-Indulgent, Trans Male Cha...",...,1/1,4.0,5.0,NaN,33,"['AllenKune', 'Sawcha', 'Azryel']",[],Izzy n'entend aucun bruit venir de l'appart qu...,Izzy n'entend aucun bruit venir de l'appart qu...,1742
3,63507442,Fluff,Le Sel sur ses Levres,['Kayukiya'],Explicit,M/M,Captive Prince - C. S. Pacat,Damen/Laurent (Captive Prince),"Damen (Captive Prince), Laurent (Captive Prince)","POV Damen (Captive Prince), Caring Damen (Capt...",...,1/1,NaN,6.0,NaN,88,"['emysterieuse', 'Shteakum', 'Azulina33']",[],\n\nLa jument a la superbe robe isabelle se d...,La jument a la superbe robe isabelle se dressa...,35140
4,63508948,Fluff,L'heureux destinataire,['Lady_Meg_666'],General Audiences,M/M,Teen Wolf (TV),Liam Dunbar/Theo Raeken,"Liam Dunbar, Theo Raeken","Letter, love letter, Romantic Fluff",...,1/1,NaN,1.0,NaN,41,['Myauada'],[],"""Je ne sais pas vraiment quoi faire. Depuis ma...","""Je ne sais pas vraiment quoi faire. Depuis ma...",2412


# Création d'une colonne où les noms des personnages sont supprimés des body

Il est possible de récupérer les noms des personnages impliqués dans chaque fanfiction à l'aide de la colonne "character" créée lors du scraping.

En explorant rapidement le corpus collecté, j'ai remarqué que les auteurs de fanfictions peuvent utiliser différentes façons d'écrire les noms des personnages.
Par convention, ils écrivent les prénoms et noms de chaque personnages séparés par des virgules. Cependant, dans certains cas, quand un même personnage possède plusieurs dénominations différentes ils peuvent utiliser le séparateur "|" pour noter ces différentes dénominations. Aussi, quand un personnage possède un surnom, ce surnom est noté entre guillemets anglais. Enfin, les auteurs peuvent ajouter des détails ou informations sur le personnages qu'ils notent entre parenthèses.

Il est donc important de prendre en compte ces différents cas de figures pour récupérer correctement les différentes dénominations des personnages de chaque fanfiction.

###Fonction qui extrait les noms des personnages à l'intérieur d'une liste

In [ ]:
# Fonction pour extraire les noms des personnages de chaque fanfiction
def clean_character_list(characters):
    if pd.isna(characters):
        return []

    # Remplacer les "|" par ","
    characters = characters.replace('|', ',')

    # Extraire les surnoms entre guillemets
    surnoms = re.findall(r'"(.*?)"', characters)

    # Supprimer les surnoms des noms principaux
    characters = re.sub(r'\s*"\s*.*?\s*"\s*', ' ', characters)

    # Supprimer les précisions entre parenthèses
    characters = re.sub(r'\s*\(.*?\)\s*', '', characters)

    # Transformer en liste propre
    character_list = [char.strip() for char in characters.split(',') if char.strip()]

    # Séparer prénom et nom pour chaque personnage
    character_list_sep_complet = []
    for char in character_list:
        # Ajouter nom et prénom séparément à la liste
        character_list_sep_complet.extend(char.split())

    # Ajouter les surnoms à la liste des noms à supprimer
    character_list_sep_complet.extend(surnoms)

    # Supprimer les doublons
    return list(set(character_list_sep_complet))

### Fonction qui supprime les noms de personnages d'un texte à partir d'une liste de ces noms

In [ ]:
# Fonction pour supprimer les noms d'un texte
def remove_characters_from_text(texte, characters):
    if pd.isna(texte) or not characters:
        return texte

    for character in characters:
        # \b pour éviter de supprimer des bouts de mots
        texte = re.sub(r'\b' + re.escape(character) + r'\b', '', texte, flags=re.IGNORECASE)

    return texte

### Récupération des noms des personnages "nettoyés" dans la colonne "characters_clean"

Je rajoute une colonne contenant la liste des noms de personnages de chaque fanfiction au dataframe à l'aide de la fonction 'clean_character_list".

In [ ]:
# Appliquer le nettoyage des noms dans une nouvelle colonne characters_clean
df_fanfic['characters_clean'] = df_fanfic['character'].apply(clean_character_list)

In [ ]:
df_fanfic[['character', 'characters_clean']].head()

,character,characters_clean
0,"Sirius Black, Remus Lupin","[Remus, Black, Sirius, Lupin]"
1,"Danny ""Danno"" Williams, Steve McGarrett","[Steve, Williams, Danny, McGarrett, Danno]"
2,"Izzy Stradlin, Axl Rose","[Rose, Axl, Stradlin, Izzy]"
3,"Damen (Captive Prince), Laurent (Captive Prince)","[Laurent, Damen]"
4,"Liam Dunbar, Theo Raeken","[Theo, Liam, Raeken, Dunbar]"


### Ajout d'une colonne body sans les noms des personnages (depuis body_clean)

J'utilise la nouvelle colonne 'characters_clean' et la fonction 'remove_characters_from_text' pour générer une nouvelle colonne 'body_no_chara' contenant le texte des fanfictions où les noms de personnages ont été supprimés.

In [ ]:
# Supprimer les noms des fanfictions dans une nouvelle colonne body_no_characters
df_fanfic['body_no_chara'] = df_fanfic.apply(lambda row: remove_characters_from_text(row['body_clean'], row['characters_clean']), axis=1)

In [ ]:
df_fanfic[['body_clean', 'body_no_chara']].head()

,body_clean,body_no_chara
0,"<< Regarde, murmure Sirius en montrant le ciel...","<< Regarde, murmure en montrant le ciel de mi..."
1,Danny deteste vraiment Steve : a peine deux jo...,deteste vraiment : a peine deux jours apres ...
2,Izzy n'entend aucun bruit venir de l'appart qu...,n'entend aucun bruit venir de l'appart quand ...
3,La jument a la superbe robe isabelle se dressa...,La jument a la superbe robe isabelle se dressa...
4,"""Je ne sais pas vraiment quoi faire. Depuis ma...","""Je ne sais pas vraiment quoi faire. Depuis ma..."


In [ ]:
# Enregistrement
df_fanfic.to_csv('data/fanfics_perso.csv', index=False)